<a href="https://colab.research.google.com/github/sanjeevram001/pdp-project/blob/main/Stockmarket_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/stock_data.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data['Sentiment'].value_counts()

In [ ]:
data['Text'][2]

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

stopwords_set = set(stopwords.words('english'))

def clean_text(text):
    """
    Remove URLs (http, https, www, t.co), HTML tags, leading numbers, and any characters that are not alphanumeric (letters and numbers) or spaces.
    Also remove extra whitespace.
    """
    if not isinstance(text, str):
        return text  # Return the original value if it's not a string

    # Remove URLs (http, https, www, t.co, etc.)
    text = re.sub(r'http[s]?://\S+|www\.\S+|t\.co/\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    # Remove leading numbers and optional whitespace
    text = re.sub(r'^\d+\s*', '', text)

    # Keep only alphanumeric characters and spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    prter = PorterStemmer()
    text = [prter.stem(word) for word in text.split() if word not in stopwords_set]

    return " ".join(text)

In [ ]:
text = data['Text'][1]
print(text)
print(clean_text(text))

In [ ]:
text = data['Text'][0]
print(type(text))

In [ ]:
data['Text'] = data['Text'].apply(lambda x : clean_text(x))

In [ ]:
data


In [ ]:
positivedata = data[data['Sentiment'] == 1]
positivedata = positivedata['Text']
negdata = data[data['Sentiment'] == -1]
negdata = negdata['Text']

import matplotlib.pyplot as plt
from collections import Counter

# Positive data
positivedata_words = ' '.join(positivedata).split()
positivedata_word_counts = Counter(positivedata_words)
positivedata_common_words = positivedata_word_counts.most_common(10)  # Display top 10 common words

# Negative data
negdata_words = ' '.join(negdata).split()
negdata_word_counts = Counter(negdata_words)
negdata_common_words = negdata_word_counts.most_common(10)  # Display top 10 common words

# Plotting
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Positive data word frequency
axes[0].barh(range(len(positivedata_common_words)), [count for _, count in positivedata_common_words], align='center')
axes[0].set_yticks(range(len(positivedata_common_words)))
axes[0].set_yticklabels([word for word, _ in positivedata_common_words])
axes[0].set_title('Positive Data Word Frequency')

# Negative data word frequency
axes[1].barh(range(len(negdata_common_words)), [count for _, count in negdata_common_words], align='center')
axes[1].set_yticks(range(len(negdata_common_words)))
axes[1].set_yticklabels([word for word, _ in negdata_common_words])
axes[1].set_title('Negative Data Word Frequency')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf=TfidfVectorizer(strip_accents=None,lowercase=True,preprocessor=None,use_idf=True,norm='l2',smooth_idf=True)
y=data.Sentiment.values
x=tfidf.fit_transform(data.Text)


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,random_state=1,test_size=0.5,shuffle=False)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500).fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [ ]:
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
print(data)

In [ ]:
news = "Tech Mahindra Share Price Updates: Tech Mahindra Closes at Rs 1585.30, Registers 4.0% Daily Gain"

In [ ]:
def prediction(comment):
    preprocessed_comment = clean_text(comment)
    comment_list = [preprocessed_comment]  # Wrap the preprocessed comment in a list
    comment_vector = tfidf.transform(comment_list)
    prediction = clf.predict(comment_vector)[0]
    return prediction



prediction = prediction(news)

In [ ]:
if prediction == 1:
    print("positive comment")
else:
    print("negative comment")